In [ ]:
import anthropic
import pandas as pd

import sqlite3
import os
from datetime import datetime

### Claude Setup

In [ ]:
api_key_file = open("./security/anthropic_api_key.txt", "r")
anthropic_api_key = api_key_file.read()

client = anthropic.Anthropic(api_key=anthropic_api_key)

In [ ]:
def query_claude(prompt, use_search=False):

    tools = []
    if use_search:
        tools = [
            {
                "name": "web_search",
                "type": "web_search_20250305",
                "max_uses": 1
            }
        ]

    return client.beta.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        temperature=1,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        tools=tools
    )

### SQLite DB Setup

In [ ]:
# Database setup - Single Company table architecture
DB_PATH = "./db/jobly.db"

def init_database():
    """Initialize the SQLite database with a single Company table"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Create single companies table with all analysis columns
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS companies (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE NOT NULL,
            is_good BOOLEAN,
            is_good_msg TEXT,
            is_good_err BOOLEAN,
            is_local BOOLEAN,
            is_local_msg TEXT,
            is_local_err BOOLEAN,
            is_hiring BOOLEAN,
            is_hiring_msg TEXT,
            is_hiring_err BOOLEAN,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()

# Initialize the database
init_database()


In [ ]:
def get_all_companies_from_db():

    all_companies_df = pd.DataFrame(columns=['id', 'name', 'is_good', 'is_good_msg', 'is_good_err', 'is_local', 'is_local_msg', 'is_local_err', 'is_hiring', 'is_hiring_msg', 'is_hiring_err', 'created_at', 'updated_at'])

    with sqlite3.connect(DB_PATH, timeout=30) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM companies")
        rows = cursor.fetchall()

        for row in rows:
            all_companies_df.loc[len(all_companies_df)] = row

    return all_companies_df

def add_new_companies_to_db(companies_to_add):

    with sqlite3.connect(DB_PATH, timeout=30) as conn:
        cursor = conn.cursor()
        for company_name in companies_to_add:
            cursor.execute("INSERT OR IGNORE INTO companies (name) VALUES (?)", (company_name,))
        conn.commit() 

def add_claude_results_to_db(result_df, bool_col_name):

    with sqlite3.connect(DB_PATH, timeout=30) as conn:
        cursor = conn.cursor()
        for _, row in result_df.iterrows():
            cursor.execute(f"UPDATE companies set is_{bool_col_name} = ?, is_{bool_col_name}_msg = ?, is_{bool_col_name}_err = ?, updated_at = CURRENT_TIMESTAMP WHERE name = ?", (row[f"IsTrue"], row[f"Message"], row[f"IsErr"], row['Company'],))
        conn.commit() 

def add_single_claude_result_to_db(company_name, is_true, message, is_err, bool_col_name):

    with sqlite3.connect(DB_PATH, timeout=30) as conn:
        cursor = conn.cursor()
        cursor.execute(f"UPDATE companies set is_{bool_col_name} = ?, is_{bool_col_name}_msg = ?, is_{bool_col_name}_err = ?, updated_at = CURRENT_TIMESTAMP WHERE name = ?", (is_true, message, is_err, company_name,))
        conn.commit() 

### Read-in new Connections CSV

In [ ]:
# connections_df = pd.read_csv("./input/Connections.csv", skiprows=2)
# connections_df = pd.read_csv("./input/Spencer_Connections_092425.csv")
connections_df = pd.read_csv("./input/Test_Connections.csv")
connections_df = connections_df[connections_df['Company'].notna()]
# print(connections_df.head())

companies_to_add = connections_df.Company.unique()
# print(companies_to_add)
print(f"Successfully read-in connections. Total unique companies: {len(companies_to_add)}")

In [ ]:
add_new_companies_to_db(companies_to_add)

In [ ]:
def loop_through_companies(companies, prompt_template, bool_col_name, use_search=False):
    
    result_df = pd.DataFrame(columns=['Company', 'IsTrue', 'Message', 'IsErr'])
    total_input_tokens = 0
    total_output_tokens = 0
    counter = 0

    for company_name in companies:

        formatted_prompt = prompt_template.format(company_name=company_name)
        message = query_claude(formatted_prompt, use_search=use_search)

        final_text = message.content[-1].text
        bool_col = False
        is_err = False

        if final_text.endswith('TRUE'):
            bool_col = True
        elif final_text.endswith('FALSE'):
            bool_col = False
        else:
            is_err = True

        result_df.loc[len(result_df)] = [company_name, bool_col, final_text, is_err]

        add_single_claude_result_to_db(company_name, bool_col, final_text, is_err, bool_col_name)

        total_input_tokens += message.usage.input_tokens
        total_output_tokens += message.usage.output_tokens
        counter += 1

        print(f"{counter:3d}/{len(companies)}: Processed company: {company_name}. Input tokens: {message.usage.input_tokens}; Output tokens: {message.usage.output_tokens}; Error: {is_err}")

    print(f"\nTotal input tokens: {total_input_tokens}; Total output tokens: {total_output_tokens}")

    return result_df

In [ ]:
# get all the companies from the db with is_good = null
companies_to_process = []
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM companies WHERE is_good IS NULL")
    companies_to_process = [row[0] for row in cursor.fetchall()]

print(f"Processing {len(companies_to_process)} new companies")

good_prompt = """
        I am looking for jobs. I am a data scientist looking for a company or non-profit working on a pro-social mission. 
        Some example cause areas: climate change, healthcare, preserving democracy, wealth inequality, education. 
        But I am interested in any others that are for the benefit of the greater good.
        Can you please let me know if this company fits that above description: {company_name}. 
        If it does meet this conditions, just reply "TRUE". If it does not, just reply "FALSE". Do not respond with the reasoning for this decision, 
        simply respond "TRUE" or "FALSE".
    """

good_result_df = loop_through_companies(companies_to_process, good_prompt, bool_col_name='good')

# add_claude_results_to_db(good_result_df, bool_col_name='good')

In [ ]:
# get all the companies from the db with is_good = true and is_local = null
companies_to_process = []
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM companies WHERE is_good IS TRUE AND is_local IS NULL")
    companies_to_process = [row[0] for row in cursor.fetchall()]

print(f"Processing {len(companies_to_process)} new companies")

local_prompt = """
        Can you please let me know if the company {company_name} is either fully remote or based in Colorado?
        I live in Colorado and can only work for a company that is based in Colorado or fully remote.
        If it is remote or in Colorado, just reply "TRUE". If it is neither, just reply "FALSE". 
        Do not respond with the reasoning for this decision, simply respond "TRUE" or "FALSE".
    """

local_result_df = loop_through_companies(companies_to_process, local_prompt, bool_col_name='local')

# add_claude_results_to_db(local_result_df, bool_col_name='local')

### Use Web Search Tool to Look at Careers Pages

In [ ]:
# get all the companies from the db with is_good = true and is_local = true and is_hiring = null
companies_to_process = []
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM companies WHERE is_good AND is_local AND is_hiring IS NULL")
    companies_to_process = [row[0] for row in cursor.fetchall()]

print(f"Processing {len(companies_to_process)} new companies")

hiring_prompt = """
        Please search the careers or jobs page of the company called {company_name} and let me know if they are hiring for a data scientist or data analystposition.
        If they are not currently hiring for such a role, just reply "FALSE". If they are hiring for such a role, reply with the link to the relevant job, and end your response with "TRUE". 
    """

hiring_result_df = loop_through_companies(companies_to_process, hiring_prompt, bool_col_name='hiring', use_search=True)

# add_claude_results_to_db(hiring_result_df, bool_col_name='hiring')

### Inspect Results

In [ ]:
get_all_companies_from_db()

In [ ]:
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT count(*) FROM companies")
    print(f"total: {cursor.fetchone()[0]}")

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT count(*) FROM companies where is_good is true")
    print(f"good: {cursor.fetchone()[0]}")

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT count(*) FROM companies where is_local is true")
    print(f"local: {cursor.fetchone()[0]}")

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT count(*) FROM companies where is_hiring is true")
    print(f"hiring: {cursor.fetchone()[0]}")

In [ ]:
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cursor.execute("SELECT name FROM companies where is_hiring is true")
    companies_to_look_at = [row[0] for row in cursor.fetchall()]

    for i, company in enumerate(companies_to_look_at):
        print(f"{i+1}: {company}")


In [ ]:
!sqlite3 jobly.db -header -csv "SELECT * FROM companies;" > snapshots/companies_$(date +%Y%m%d).csv

In [ ]:
# backup_df = pd.read_csv("./snapshots/companies_20250925.csv")
# # convert created_at to datetime
# backup_df['created_at'] = pd.to_datetime(backup_df['created_at'])
# backup_df['updated_at'] = pd.to_datetime(backup_df['updated_at'])
# display(backup_df.head())

# # show types of columns
# backup_df.dtypes

# for index, row in backup_df.iterrows():
#     # Convert pandas Timestamp objects to strings for SQLite compatibility
#     created_at_str = row['created_at'].strftime('%Y-%m-%d %H:%M:%S') if pd.notna(row['created_at']) else None
#     updated_at_str = row['updated_at'].strftime('%Y-%m-%d %H:%M:%S') if pd.notna(row['updated_at']) else None
    
#     with sqlite3.connect(DB_PATH, timeout=30) as conn:
#         cursor = conn.cursor()
#         cursor.execute("INSERT INTO companies (name, is_good, is_good_msg, is_good_err, is_local, is_local_msg, is_local_err, is_hiring, is_hiring_msg, is_hiring_err, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (row['name'], row['is_good'], row['is_good_msg'], row['is_good_err'], row['is_local'], row['is_local_msg'], row['is_local_err'], row['is_hiring'], row['is_hiring_msg'], row['is_hiring_err'], created_at_str, updated_at_str))
#         print(f"Inserted row {index + 1}")